In [1]:
from src.data_utils.dataset import build_datset_train_val
from src.Transformers.transformers_prototype import build_transformer_model
from src.callbacks import get_predefine_callbacks
import optuna
import tensorflow as tf

2023-08-19 05:18:40.787395: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-19 05:18:41.432000: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-19 05:18:42.181967: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-19 05:18:42.202957: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [2]:
train_dataset, val_dataset = build_datset_train_val(split=0.8, batch_size=256)

train split: 28160 | val split: 6656


In [3]:
def objective(trial):
    tf.keras.backend.clear_session()
    
    model = build_transformer_model(trial=trial)
    model.fit(train_dataset, validation_data=val_dataset, epochs=20, callbacks=get_predefine_callbacks(model_name="v2"))

    levenshtein = model.evaluate(val_dataset)[-1]
    return  levenshtein

In [4]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50, gc_after_trial=True, show_progress_bar=True)

[I 2023-08-19 05:18:44,735] A new study created in memory with name: no-name-8bebbe6d-0542-445d-8dba-d65063b28757


  0%|          | 0/50 [00:00<?, ?it/s]

/home/matias/pythonenv/fingerspelling/lib/python3.10/site-packages/optuna/distributions.py:700: UserWarning: The distribution is specified by [1, 8] and step=2, but the range is not divisible by `step`. It will be replaced by [1, 7].
  warnings.warn(


Epoch 1/20


2023-08-19 05:18:49.644500: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8700
2023-08-19 05:18:50.128524: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-08-19 05:18:50.656763: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x555db2590300 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-19 05:18:50.656791: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 SUPER, Compute Capability 7.5
2023-08-19 05:18:50.663004: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-19 05:18:50.734872: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-08-19 05:18:50.788726: I ./tensorflow/compiler/jit/devic

    212/Unknown - 45s 169ms/step - loss: 7.1218 - accuracy: 0.3410 - sparse_levenshtein: inf

2023-08-19 05:19:30.920247: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 10394615387728924976
2023-08-19 05:19:30.920273: I tensorflow/core/framework/local_rendezvous.cc:409] Local rendezvous send item cancelled. Key hash: 13099105851638699190
2023-08-19 05:19:30.920280: I tensorflow/core/framework/local_rendezvous.cc:409] Local rendezvous send item cancelled. Key hash: 11851479576935725475
2023-08-19 05:19:30.920295: I tensorflow/core/framework/local_rendezvous.cc:409] Local rendezvous send item cancelled. Key hash: 4544500555251088065
2023-08-19 05:19:30.920305: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 106548396391659787
2023-08-19 05:19:30.920312: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 8329285869065638153
2023-08-19 05:19:30.920335: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv i

212/212 [==============================] - 50s 191ms/step - loss: 7.1218 - accuracy: 0.3410 - sparse_levenshtein: inf - val_loss: 2.0401 - val_accuracy: 0.4465 - val_sparse_levenshtein: 0.9117
Epoch 2/20
212/212 [==============================] - 38s 181ms/step - loss: 1.9210 - accuracy: 0.4453 - sparse_levenshtein: 0.8951 - val_loss: 1.8747 - val_accuracy: 0.4550 - val_sparse_levenshtein: 0.8925
Epoch 3/20
212/212 [==============================] - 38s 181ms/step - loss: 1.9032 - accuracy: 0.4494 - sparse_levenshtein: 0.8903 - val_loss: 1.8730 - val_accuracy: 0.4546 - val_sparse_levenshtein: 0.8947
Epoch 4/20
212/212 [==============================] - 39s 184ms/step - loss: 1.8707 - accuracy: 0.4545 - sparse_levenshtein: 0.8856 - val_loss: 1.8625 - val_accuracy: 0.4610 - val_sparse_levenshtein: 0.9171
Epoch 5/20
212/212 [==============================] - 40s 188ms/step - loss: 1.8693 - accuracy: 0.4566 - sparse_levenshtein: 0.8836 - val_loss: 1.9212 - val_accuracy: 0.4458 - val_sparse

In [ ]:
tf.keras.backend.clear_session()

trials = study.best_trials

for index, trial in enumerate(trials):
    print(f"Best model: {index+1}")
    model_name = "v2"

    model = build_gru_model(trial=trial)
    model.fit(train_dataset, validation_data=val_dataset, epochs=5000, callbacks=get_predefine_callbacks(model_name=model_name))
    print(model.summary())

    print('validation levenshtein distance: {}'.format(trial.value))
    print("Best hyperparameters: {}".format(trial.params))

    model.load_weights(f"../best_model/prototype/{model_name}")

    print(f"Metrics in Validation: {model.evaluate(val_dataset)}")